# **CHAPTER 9**
# **Unsupervised Learning Techniques**

Unsupervised learning deals with datasets where input features are available, but no labels are provided. Unlike supervised learning, which relies heavily on labeled data, unsupervised learning explores hidden patterns and structures directly from raw data. Since most real-world data is unlabeled, unsupervised learning holds significant potential. As Yann LeCun famously stated, unsupervised learning forms the foundation of intelligence, while supervised and reinforcement learning build upon it.
This chapter focuses on three main unsupervised learning tasks: clustering, anomaly detection, and density estimation. These techniques are useful in many domains, such as manufacturing defect detection, customer segmentation, data exploration, and image analysis. The chapter builds upon dimensionality reduction techniques discussed previously and expands into more advanced unsupervised methods.

**Clustering: Concept and Applications**

Clustering is the task of grouping similar instances together without predefined labels. The goal is to identify natural groupings in data based on similarity measures. Unlike classification, clustering does not rely on known class labels and instead discovers structure directly from the data.
Clustering is widely used in customer segmentation, exploratory data analysis, anomaly detection, recommender systems, search engines, image segmentation, and semi-supervised learning. Different clustering algorithms define clusters differently: some rely on centroids, others on density or hierarchical structures. Because there is no universal definition of a “cluster,” different algorithms can produce different results on the same dataset.

**K-Means Clustering**

K-Means is one of the most popular clustering algorithms due to its simplicity, speed, and scalability. It works best on datasets where clusters are roughly spherical, similar in size, and well separated.
The algorithm requires specifying the number of clusters kin advance. Each instance is assigned to the nearest centroid, and centroids are iteratively updated until convergence.




In [10]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans

In [11]:
cal_housing = fetch_california_housing(as_frame=True)
housing = cal_housing.frame  # dataframe

In [12]:
imputer = SimpleImputer(strategy="median")
X = imputer.fit_transform(housing)  # semua kolom numerik

In [13]:
k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(X)

In [14]:
print(y_pred[:10])

[0 2 0 0 0 0 0 0 0 2]


**Hard vs Soft Clustering**

Standard K-Means performs hard clustering, where each instance belongs to exactly one cluster. However, in some cases, it is useful to measure how close an instance is to each cluster.
Soft clustering assigns a score to each instance for every cluster, often using distance or similarity measures. K-Means supports this through the transform() method, which computes distances to all centroids. This approach can also be used as a nonlinear dimensionality reduction technique by transforming data into a lower-dimensional space of cluster distances.

**The K-Means Algorithm and Initialization**

K-Means works by alternating between two steps: assigning instances to the nearest centroid and updating centroids based on cluster means. The algorithm is guaranteed to converge but may converge to a local optimum depending on centroid initialization.
To mitigate poor initialization, K-Means is typically run multiple times using different initial centroids. The best solution is selected based on inertia, which measures the mean squared distance between instances and their closest centroids.
K-Means++ improves initialization by spreading centroids apart, significantly reducing the risk of suboptimal solutions. Scikit-Learn uses K-Means++ by default.


**Accelerated and Mini-Batch K-Means**

Accelerated K-Means reduces computational cost by avoiding unnecessary distance calculations using the triangle inequality. Mini-Batch K-Means further improves scalability by updating centroids using small random subsets of data.


In [15]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

MiniBatchKMeans(n_clusters=5)

**Choosing the Number of Clusters**

Selecting the optimal number of clusters is a critical challenge. Inertia always decreases as kincreases, making it unreliable alone. The elbow method identifies a point where inertia decreases more slowly.
The silhouette score provides a more informative metric by evaluating how well instances fit within their clusters compared to others. Higher silhouette scores indicate better-defined clusters.


**Limitations of K-Means**

Despite its advantages, K-Means has several limitations. It struggles with clusters of varying sizes, densities, and non-spherical shapes. It is also sensitive to feature scaling and outliers. In such cases, alternative clustering methods like Gaussian Mixture Models or DBSCAN often perform better.


**Image Segmentation with K-Means**

K-Means can be applied to image segmentation by clustering pixel colors. Each pixel is treated as a data point in RGB space, and similar colors are grouped together.


In [19]:
from skimage import data
image = data.astronaut()  # contoh gambar RGB
print(image.shape)


(512, 512, 3)


In [20]:
X = image.reshape(-1, 3)
kmeans = KMeans(n_clusters=8).fit(X)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)

**Clustering for Preprocessing**

Clustering can improve supervised learning by serving as a preprocessing step. By transforming instances into distances to cluster centroids, models can achieve higher accuracy.
Using K-Means before Logistic Regression on the digits dataset significantly improves classification performance, especially when the number of clusters is optimized using cross-validation.


In [21]:
from sklearn.datasets import load_digits
X_digits, y_digits = load_digits(return_X_y=True)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)

In [23]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [24]:
log_reg.score(X_test, y_test)

0.9688888888888889

In [25]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=50)),
    ("log_reg", LogisticRegression()),
])
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                ('log_reg', LogisticRegression())])

In [26]:
pipeline.score(X_test, y_test)

0.9577777777777777

In [27]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

[CV] END ...............................kmeans__n_clusters=2; total time=   0.2s
[CV] END ...............................kmeans__n_clusters=2; total time=   0.2s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

[CV] END ...............................kmeans__n_clusters=2; total time=   0.2s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.2s


...

[CV] END ..............................kmeans__n_clusters=99; total time=   3.2s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ..............................kmeans__n_clusters=99; total time=   1.6s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                                       ('log_reg', LogisticRegression())]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [28]:
grid_clf.best_params_

{'kmeans__n_clusters': 90}

In [29]:
grid_clf.score(X_test, y_test)

0.9644444444444444

**Semi-Supervised Learning with Clustering**

Clustering is valuable when labeled data is scarce. Representative instances closest to cluster centroids can be manually labeled, yielding better performance than labeling random instances.
Label propagation extends these labels to nearby instances, further improving accuracy. By only propagating labels to instances closest to centroids, the model achieves strong performance with minimal labeled data.


In [30]:
n_labeled = 50
log_reg = LogisticRegression()
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

LogisticRegression()

In [31]:
log_reg.score(X_test, y_test)

0.8422222222222222

In [32]:
k = 50
kmeans = KMeans(n_clusters=k)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

In [33]:
y_representative_digits = np.array([4, 8, 0, 6, 8, 3, ..., 7, 6, 2, 3, 1, 1])

In [37]:
print(X_representative_digits.shape)
print(y_representative_digits.shape)


(50, 64)
(13,)


In [41]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.09333333333333334

In [42]:
percentile_closest = 20

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [43]:
log_reg = LogisticRegression()
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.07111111111111111

In [44]:
log_reg.score(X_test, y_test)

0.07111111111111111

**Active Learning**

Active learning enhances semi-supervised learning by iteratively selecting the most informative unlabeled instances for human labeling. Common strategies include uncertainty sampling and disagreement-based methods, allowing efficient improvement of model performance with minimal labeling effort.


**DBSCAN**

DBSCAN is a density-based clustering algorithm that identifies clusters as dense regions separated by low-density areas. It can detect clusters of arbitrary shapes and automatically identifies anomalies.


In [45]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.05)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [46]:
dbscan.labels_

array([ 0, -1,  2,  1,  2,  1,  3,  0, -1,  1,  0,  0,  4,  3,  3,  0,  3,
        1,  1,  5,  4,  3,  1,  2,  6,  0,  2,  7,  1,  2,  8,  3,  2,  5,
        1,  1,  0,  7,  6,  6,  8,  4,  3,  3,  3,  6,  3,  3,  2,  1,  4,
        6,  3,  3,  4,  7,  1,  1,  6,  8,  1,  0,  4,  2,  2,  3,  7,  6,
        6,  6,  6,  3,  1,  6,  4,  1,  0, -1,  0,  6,  3,  6,  1,  2,  3,
        4,  8,  8,  3,  0,  7,  2, -1,  2, -1,  4,  1,  7,  2,  1,  2,  1,
        1,  0, -1,  1,  1,  1,  2,  0,  1,  1,  1,  8,  8, 11,  6,  2,  4,
        6,  6,  1,  6,  3,  1,  2,  0,  6,  5,  6,  2,  0,  7,  7,  1, -1,
        3,  2,  6,  6,  2, -1,  1,  0,  1,  6, -1,  1,  6,  0,  0,  8,  0,
        0,  1,  2,  6,  0,  1,  0,  2,  0,  6,  2,  4,  3,  9,  6,  1,  3,
        6,  8,  1,  7,  6,  1,  6,  4,  2,  2,  6,  1,  2,  6,  2,  2,  4,
        8,  4,  2,  4,  1, 11,  2,  1,  2,  3,  4,  6,  2,  8,  0,  3,  2,
        1, -1,  3,  0,  1,  6,  0,  1,  2, -1,  2,  7,  3,  1,  4,  1, 10,
        8,  1,  2,  2, 10

In [47]:
len(dbscan.core_sample_indices_)

777

In [48]:
dbscan.core_sample_indices_

array([  0,   3,   4,   5,   6,   7,   9,  10,  12,  13,  14,  15,  16,
        17,  18,  19,  20,  21,  23,  24,  25,  27,  28,  29,  30,  31,
        32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  44,  45,
        46,  47,  48,  49,  50,  51,  52,  54,  55,  56,  57,  58,  59,
        61,  62,  63,  64,  65,  66,  67,  69,  70,  71,  72,  73,  74,
        76,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  93,  95,  96,  97,  98,  99, 100, 101, 102, 103, 106,
       107, 108, 109, 111, 112, 113, 114, 116, 117, 118, 120, 121, 122,
       124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 137, 138, 139,
       140, 143, 144, 145, 147, 148, 149, 151, 152, 153, 154, 155, 157,
       158, 161, 163, 165, 166, 167, 168, 170, 171, 172, 174, 175, 176,
       177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190,
       191, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 204, 206,
       207, 208, 209, 210, 211, 212, 214, 216, 217, 218, 219, 22

In [49]:
dbscan.components_

array([[ 1.84343509,  0.01685793],
       [ 1.33815722, -0.50226508],
       [ 0.81172911,  0.69061349],
       ...,
       [ 0.94800194,  0.24172864],
       [ 1.36161755, -0.40858626],
       [-0.98626368,  0.10778478]])

In [50]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

KNeighborsClassifier(n_neighbors=50)

In [51]:
X_new = np.array([[-0.5, 0], [0, 0.5], [1, -0.1], [2, 1]])
knn.predict(X_new)

array([8, 6, 7, 0])

In [52]:
knn.predict_proba(X_new)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.26, 0.2 , 0.  , 0.54, 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.44, 0.  , 0.  , 0.  , 0.  , 0.  , 0.56, 0.  , 0.  , 0.  ,
        0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ]])

In [53]:
y_dist, y_pred_idx = knn.kneighbors(X_new, n_neighbors=1)
y_pred = dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx]
y_pred[y_dist > 0.2] = -1
y_pred.ravel()

array([-1,  6,  7, -1])

**Other Clustering Algorithms**

The chapter briefly introduces additional clustering methods such as Agglomerative Clustering, BIRCH, Mean-Shift, Affinity Propagation, and Spectral Clustering. Each method has unique strengths and trade-offs in terms of scalability, flexibility, and computational cost.


**Gaussian Mixture Models (GMM)**

Gaussian Mixture Models assume data is generated from multiple Gaussian distributions. Unlike K-Means, GMMs support soft clustering and can model clusters with different shapes, sizes, and orientations.


In [54]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10)

In [55]:
gm.weights_

array([0.20092227, 0.59167259, 0.20740514])

In [56]:
gm.means_

array([[ 1.75671554, -0.04894254],
       [ 0.50738134,  0.24404991],
       [-0.72959216,  0.56802199]])

In [57]:
gm.covariances_

array([[[ 0.04749525,  0.05895939],
        [ 0.05895939,  0.08758032]],

       [[ 0.16961221, -0.10342078],
        [-0.10342078,  0.29377095]],

       [[ 0.05575235,  0.06360406],
        [ 0.06360406,  0.08753502]]])

In [58]:
gm.converged_

True

In [59]:
gm.n_iter_

15

In [60]:
gm.predict(X)

array([0, 1, 1, 0, 1, 0, 2, 0, 2, 1, 0, 0, 1, 2, 2, 0, 2, 1, 0, 2, 1, 2,
       0, 1, 1, 0, 1, 1, 1, 1, 2, 2, 1, 2, 0, 1, 0, 1, 1, 1, 2, 1, 2, 2,
       2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 0, 0, 1, 2, 1, 0, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 0, 0, 1, 2, 1, 1, 1, 2, 1, 2, 2,
       2, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 0, 1, 2, 1, 1, 0,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 2, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0, 1, 1, 1, 2, 1,
       1, 1, 2, 0, 2, 1, 1, 2, 2, 0, 0, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1, 0,
       1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 2,
       2, 1, 2, 0, 1, 2, 2, 1, 1, 2, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 2, 1, 2,
       2, 0, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 0, 2,

In [61]:
gm.predict_proba(X)

array([[9.98902657e-001, 1.09734316e-003, 9.03947031e-203],
       [2.73070107e-001, 7.26929893e-001, 1.71101104e-178],
       [3.91481872e-196, 8.79440588e-001, 1.20559412e-001],
       ...,
       [7.09815744e-001, 2.90184256e-001, 6.46624562e-181],
       [7.07295129e-226, 2.48167220e-004, 9.99751833e-001],
       [2.44585744e-264, 2.77170639e-002, 9.72282936e-001]])

In [62]:
X_new, y_new = gm.sample(6)
X_new

array([[ 1.72673487,  0.00417065],
       [ 1.44119757, -0.37212293],
       [ 1.65770023,  0.01490106],
       [ 2.12996568,  0.38696485],
       [ 0.84543569,  0.17115128],
       [ 0.62134002,  0.23918485]])

In [63]:
y_new

array([0, 0, 0, 0, 1, 1])

In [64]:
gm.score_samples(X)

array([ 6.25411399e-02, -2.23590777e+00, -2.21626726e+00, -1.50957482e+00,
       -1.88134029e+00, -3.91874614e-01, -1.05141405e+00, -3.63276278e+00,
       -2.33607798e+00, -1.75853963e+00, -5.15123048e-01, -3.76050471e+00,
       -1.68955339e+00, -1.60190889e+00, -6.09617741e-01, -1.38776528e+00,
       -8.37731240e-01, -2.07559263e+00, -4.00739863e-01, -1.87604886e+00,
       -1.62084328e+00, -3.82092508e-02, -4.61029338e-01, -1.93150101e+00,
       -1.64652042e+00,  1.60804130e-01, -1.48187846e+00, -1.33956970e+00,
       -1.60721139e+00, -1.44479397e+00, -3.86017446e-02, -3.02821780e-01,
       -1.79311102e+00, -1.98839648e+00, -4.00624600e-01, -2.34277332e+00,
       -4.23150494e-01, -1.27368962e+00, -1.83693446e+00, -1.44487220e+00,
       -7.59251173e-01, -1.33080634e+00, -9.49985717e-01, -1.52410377e+00,
       -4.38886112e-01, -1.46371950e+00, -2.45599100e-01, -2.21988429e-01,
       -1.81394066e+00, -1.86027228e+00, -1.52414761e+00, -1.51276123e+00,
       -9.48817780e-02,  

**Anomaly Detection and Model Selection**

Low-density regions identified by GMMs can be treated as anomalies. Thresholds can be adjusted based on acceptable false positive or false negative rates.
To choose the number of clusters, information criteria such as BIC and AIC are used instead of inertia or silhouette scores. These criteria balance model complexity and data fit.


In [65]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4)
anomalies = X[densities < density_threshold]